In [1]:
## Import libraries

import pandas as pd
import re
import datetime
import requests
import bs4
import numpy as np

In [2]:
supermarket_csv = ["aldi_products.csv", "aldisued_products.csv", "edeka_products.csv", "kaufland_products.csv", "rewe_products.csv", "lidl_products.csv"]

In [14]:
def data_cleaner(supermarket_input):
    
        
    df = pd.read_csv(supermarket_input) 
    
    aldi_df = df
    
    aldi_df["Price"] = aldi_df["Price"].str.replace(" €","").str.replace(",",".")
    aldi_df["Price per"] = aldi_df["Price per"].str.replace(" €","").str.replace(",",".")
    aldi_df["Price"] = pd.to_numeric(aldi_df['Price'], errors='coerce')
    aldi_df["Price per"] = pd.to_numeric(aldi_df['Price per'], errors='coerce')
    
        
    aldi_df["1kg Div"] = np.where(aldi_df["Unit"] == ("Preis pro 1 KG"),(aldi_df["Price per"])/10,aldi_df["Price per"])
    aldi_df["ML Div "] = np.where(aldi_df["Unit"] == ("Preis pro 1 Liter"),(aldi_df["Price per"])/10,aldi_df["Price per"])
    aldi_df["Div_merged"] = 0
    aldi_df["Div_merged"] = np.where(aldi_df["Unit"].str.contains("G"), aldi_df["1kg Div"],0)
    aldi_df["Div_merged2"] = np.where(aldi_df["Unit"].str.contains("Liter","ml"), aldi_df["ML Div "], aldi_df["Div_merged"])
    aldi_df["Div_merged3"] = np.where(aldi_df["Unit"].str.contains("ml"), aldi_df["ML Div "], aldi_df["Div_merged2"])
    aldi_df["Comparable Price"] = np.where(aldi_df["Div_merged3"] == 0, aldi_df["Price per"], aldi_df["Div_merged3"])
    
    
    dropped = ["Unnamed: 0","1kg Div","ML Div ","Div_merged","Div_merged2","Div_merged3", "Price per"]
    aldi_df = aldi_df.drop(columns=dropped)
    
    aldi_df["Unit"] = aldi_df["Unit"].str.replace("Preis pro 1 KG","100 Gramm").str.replace("Preis pro 100 Gramm", "100 Gramm").str.replace("Preis pro 1 Liter", "100 ml").str.replace("Preis pro 100 ml", "100 ml")
            
    aldi_df["Packet Size"] = aldi_df["Packet Size"].str.replace("(","").str.replace(")","")
        
        
    
        
    
    return aldi_df
        
        
        
        
    
        
        
        

        
    

    
    
    

In [15]:
for i in supermarket_csv:
    frame = data_cleaner(i)
    csvpath = "cleaned_" + i
    frame.to_csv(csvpath)

In [13]:
data_cleaner(supermarket_csv)

,Name,Price,Unit,Packet Size,Supermarket,Comparable Price
0,Coca Cola,0.99,100 ml,Pet-Flasche 1.25 L,lidl,0.079
1,Dulano Delikatess Wiener Würstchen knackig und...,2.79,100 Gramm,Packung 400 Gramm,lidl,0.698
2,Gebirgsjäger Delikatess Frikadellen 5 Stück,1.99,100 Gramm,Packung 500 Gramm,lidl,0.398
3,Nutella,3.99,100 Gramm,Vorteils Glas 1000 Gramm,lidl,0.399
4,Master Crumble Schoko-Knusper,1.79,100 Gramm,Packung 750 Gramm,lidl,0.239
...,...,...,...,...,...,...
5483,Veltins V+ Curuba,2.79,100 Gramm,"XXL Packung 200 Gramm, ca. 50 Scheiben",lidl,1.400
5484,Valensina Frühstücks Nektar Orange Mango Honig,0.88,100 ml,Pet-Flasche 1 L,lidl,0.088
5485,MM Extra Sekt Rose trocken,2.69,100 ml,Flasche Glas 0.75 L,lidl,0.359
5486,Arla Svenso geschnitten,1.49,100 Gramm,Packung 150 Gramm,lidl,0.990
